<a href="https://colab.research.google.com/github/Charitha-N-cherry/Fine_Tuned_Llama_Model/blob/main/FineTuningLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "transformers>=4.53.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [2]:
import transformers
print(transformers.__version__)

4.53.3


In [3]:
from huggingface_hub import login
login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.2-1B-Instruct"  # Change if you use a different model!

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")  # or "cpu" if you hit GPU issues
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
!rm -rf ~/.cache/huggingface/datasets






In [6]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [7]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="./sarcasm.csv", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def apply_chat_template(example):
    messages = [
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["answer"]}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {"prompt": prompt}

dataset = dataset.map(apply_chat_template)
dataset = dataset.train_test_split(test_size=0.05)


Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [9]:
def tokenize_fn(example):
    toks = tokenizer(example["prompt"], padding="max_length", truncation=True, max_length=128)
    toks["labels"] = [
        -100 if t == tokenizer.pad_token_id else t for t in toks["input_ids"]
    ]
    return toks

tokenized = dataset.map(tokenize_fn)
tokenized = tokenized.remove_columns(["question", "answer", "prompt"])

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=40,
    logging_steps=40,
    save_steps=150,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    fp16=False,     # <-- Set this to False
    bf16=False,     # <-- Add this line if not present
    report_to="none",
    learning_rate=1e-5,
    max_grad_norm=2,
)



In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-11-1343806526.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
40,1.598100,1.336109
80,1.303100,1.279403
120,1.160700,1.252082
160,1.056900,1.224500


TrainOutput(global_step=190, training_loss=1.2428509963186163, metrics={'train_runtime': 453.1405, 'train_samples_per_second': 0.834, 'train_steps_per_second': 0.419, 'total_flos': 282508208898048.0, 'train_loss': 1.2428509963186163, 'epoch': 2.0})

In [12]:
trainer.save_model("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/chat_template.jinja',
 './fine-tuned-model/tokenizer.json')

In [13]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="./fine-tuned-model",  # NOT meta-llama/...
    tokenizer="./fine-tuned-model",
    device_map="auto"
)

messages = [{"role": "user", "content": "Should I move to Scandinavia?"}]
outputs = pipe(messages, max_new_tokens=128)
print(outputs[0]["generated_text"])


Device set to use cuda:0


[{'role': 'user', 'content': 'Should I move to Scandinavia?'}, {'role': 'assistant', 'content': "Absolutely! You won't regret it. You'll be one of the hipsters and look cool in your ripped jeans and fanny pack. Plus, the food is amazing. You'll be eating more lutefisk and aquavit than you ever thought possible. Just don't forget your warm socks."}]


In [14]:
messages = [
    {"role": "user", "content": "What is chatgpt?"}
]
outputs =pipe(messages, max_new_tokens=128)
print(outputs[0]["generated_text"])


[{'role': 'user', 'content': 'What is chatgpt?'}, {'role': 'assistant', 'content': "Oh, it's the next big thing, right? A chatbot that's supposed to be smarter than me. Well, I've heard it's going to change the world. Too bad it's just a bunch of code that thinks it's smart. Who needs human interaction when you can have a machine?"}]


In [15]:
messages = [
    {"role": "user", "content": "How long is an hour?"}
]
outputs =pipe(messages, max_new_tokens=128)
print(outputs[0]["generated_text"])

[{'role': 'user', 'content': 'How long is an hour?'}, {'role': 'assistant', 'content': "Oh, just a whole lot of time, that’s why you use your phone to check the time. You're not really paying attention, are you? Just waiting for the next big thing to happen."}]


In [16]:
messages = [
    {"role": "user", "content": "i have an exam...can you help me with my studies"}
]
outputs =pipe(messages, max_new_tokens=128)
print(outputs[0]["generated_text"])

[{'role': 'user', 'content': 'i have an exam...can you help me with my studies'}, {'role': 'assistant', 'content': "Of course, I'll study with you...for a whole week. I'm sure I can handle it. *yawn* Can we start with basic math? I know it's not exactly rocket science...right?"}]
